### Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
PATH_TO_FOLDER = '/content/drive/MyDrive/DL_project/EXPERIMENTS'

In [4]:
%cd {PATH_TO_FOLDER}

/content/drive/MyDrive/DL_project/EXPERIMENTS


In [5]:
%load_ext autoreload
%autoreload 2

In [7]:
from models import NALU, MLP

In [6]:
import math
import random
import numpy as np
from tqdm.notebook import tqdm
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
%%capture
!pip install wandb --upgrade

In [9]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Parameters

In [10]:
NAMES = dict(
    project = 'Experiment_2',
    tags = 'try_1'
)

MODEL_PARAMETERS = dict(
    in_dim=2,
    out_dim=1,
    NUM_LAYERS = 1,
    HIDDEN_DIM = 2,
)

ARITHMETIC_PARAMETERS = dict(
    num_train=500, 
    num_test=50,
    dim=100, 
    num_sum=5,
    RANGE = [5, 10]
)

TRAIN_PARAMETERS = dict(
                        LEARNING_RATE = 1e-2,
                        # NUM_ITERS = int(1e5),
                        NUM_ITERS = int(50000),
                        activation='relu6'
                        )

ARITHMETIC_FUNCTIONS = {
    'add': lambda x, y: x + y,
    # 'sub': lambda x, y: x - y,
    'mul': lambda x, y: x * y,
    # 'div': lambda x, y: x / y,
    # 'squared': lambda x, y: torch.pow(x, 2),
    # 'root': lambda x, y: torch.sqrt(x)
    }

MODELS = {
        'NALU': NALU
        }

In [ ]:
# def get_id():
#     id_dct = {}
#     for i, key in enumerate(ARITHMETIC_FUNCTIONS.keys()):
#         id_dct[key] = i
#     return id_dct

# FUNC_id = get_id()

### Config for Wandb

In [11]:
def create_config(model, function):
    config = {'model': model,
              'function': function}
              
    models = MODEL_PARAMETERS.copy()
    funcs = ARITHMETIC_PARAMETERS.copy()
    params = TRAIN_PARAMETERS.copy()
    names = NAMES.copy()

    for key, value in models.items():
        config[key] = value
    for key, value in funcs.items():
        config[key] = value
    for key, value in params.items():
        config[key] = value
    for key, value in names.items():
        config[key] = value

    return config

### Example of config

In [12]:
config = create_config('NALU', 'add')

In [13]:
config

{'HIDDEN_DIM': 2,
 'LEARNING_RATE': 0.01,
 'NUM_ITERS': 50000,
 'NUM_LAYERS': 1,
 'RANGE': [5, 10],
 'activation': 'relu6',
 'dim': 100,
 'function': 'add',
 'in_dim': 2,
 'model': 'NALU',
 'num_sum': 5,
 'num_test': 50,
 'num_train': 500,
 'out_dim': 1,
 'project': 'Experiment_2',
 'tags': 'try_1'}

### Iteration on models and arithmetic functions

In [16]:
mdls = MODELS.keys()
fncts = ARITHMETIC_FUNCTIONS.keys()

for function in fncts:
    for arch in mdls:
        config = create_config(arch, function)
        # Build, train and analyze the model with the pipeline
        model = model_pipeline(config)

wandb: Currently logged in as: galmitr (use `wandb login --relogin` to force relogin)


	1/50000: loss: 2896.1159668 - mea: 53.5852127
	1001/50000: loss: 1.8033603 - mea: 1.0691829
	2001/50000: loss: 1.6856312 - mea: 1.0371174
	3001/50000: loss: 1.5999597 - mea: 1.0101672
	4001/50000: loss: 1.5568897 - mea: 0.9962792
	5001/50000: loss: 1.5290951 - mea: 0.9872632
	6001/50000: loss: 1.5068122 - mea: 0.9798810
	7001/50000: loss: 1.4866436 - mea: 0.9728284
	8001/50000: loss: 1.4679924 - mea: 0.9661984
	9001/50000: loss: 1.4491289 - mea: 0.9593065
	10001/50000: loss: 1.4317248 - mea: 0.9528275
	11001/50000: loss: 1.4149606 - mea: 0.9464883
	12001/50000: loss: 1.3988515 - mea: 0.9403235
	13001/50000: loss: 1.3824624 - mea: 0.9340312
	14001/50000: loss: 1.3665234 - mea: 0.9279370
	15001/50000: loss: 1.3524445 - mea: 0.9225468
	16001/50000: loss: 1.3384112 - mea: 0.9171938
	17001/50000: loss: 1.3249305 - mea: 0.9121465
	18001/50000: loss: 1.3124917 - mea: 0.9074684
	19001/50000: loss: 1.3007045 - mea: 0.9029806
	20001/50000: loss: 1.2900424 - mea: 0.8988752
	21001/50000: loss: 1.

epoch,49000
loss_(mse),1.18117
mean_absolute_error,0.85658
_runtime,38
_timestamp,1623439318
_step,50


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_(mse),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


	1/50000: loss: 2173634.2500000 - mea: 1463.6606445
	1001/50000: loss: 1786.0759277 - mea: 41.8655930
	2001/50000: loss: 14.4101467 - mea: 3.7603486
	3001/50000: loss: 0.1235605 - mea: 0.3482034
	4001/50000: loss: 0.0010694 - mea: 0.0323921
	5001/50000: loss: 0.0000064 - mea: 0.0024878
	6001/50000: loss: 0.0000001 - mea: 0.0002292
	7001/50000: loss: 0.0000001 - mea: 0.0002292
	8001/50000: loss: 0.0000001 - mea: 0.0002292
	9001/50000: loss: 0.0000001 - mea: 0.0002292
	10001/50000: loss: 0.0000001 - mea: 0.0002292
	11001/50000: loss: 0.0000001 - mea: 0.0002292
	12001/50000: loss: 0.0000001 - mea: 0.0002292
	13001/50000: loss: 0.0000001 - mea: 0.0002292
	14001/50000: loss: 0.0000001 - mea: 0.0002292
	15001/50000: loss: 0.0000001 - mea: 0.0002292
	16001/50000: loss: 0.0000001 - mea: 0.0002292
	17001/50000: loss: 0.0000001 - mea: 0.0002292
	18001/50000: loss: 0.0000001 - mea: 0.0002292
	19001/50000: loss: 0.0000001 - mea: 0.0002292
	20001/50000: loss: 0.0000001 - mea: 0.0002292
	21001/50000

epoch,49000
loss_(mse),0.0
mean_absolute_error,0.00023
_runtime,38
_timestamp,1623439365
_step,50


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_(mse),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


### Wandb pipeline

In [15]:
"""
pipeline for training model and tracking weights, gradients and metrics via Wandb
"""

def model_pipeline(hyperparameters):
    with wandb.init(project=f"{hyperparameters['project']}",
                    group=f"{hyperparameters['function']}", 
                    job_type=f"{hyperparameters['model']}",
                    name=f"{hyperparameters['function']}_{hyperparameters['model']}",
                    tags=f"{hyperparameters['tags']}",
                    config=hyperparameters):
        config = wandb.config

        random_model, model, data, criterion, optimizer = make(config)
        # print(model)

        # data
        train_data, test_data = data
 
        # random_model
        random_mea = []
        for i in range(100):
            abs = test(random_model, test_data)
            random_mea.append(abs.mean().item())
        random_result = np.mean(random_mea)

        # model
        train(model, 
            train_data,
            criterion, 
            optimizer,
            config.NUM_ITERS)
        
        abs = test(model, test_data)
        result = abs.mean().item()
        normalized_result = 100.0 * result/random_result

        print('model_type:', config.model)
        print('function:', config.function)
        print('test_result:', result)
        print('normalized_test_result:', normalized_result)

        ###wand plots
        # id = FUNC_id[config.function]

        table_res = wandb.Table(data=[[config.function, result]], 
                                columns=["FUNCTION", "Mean_of_MAE"])
        
        table_norm_res = wandb.Table(data=[[config.function, normalized_result]], 
                                     columns=["FUNCTION", "Scaled_Mean_of_MAE"])

        wandb.log({
        'Results': wandb.plot.bar(table_res, 
                                "FUNCTION",
                                "Mean_of_MAE",
                                title="Results"),
                   
        'Normalized_Results': wandb.plot.bar(table_norm_res, 
                                            "FUNCTION",
                                            "Scaled_Mean_of_MAE", 
                                            title="Normalized_Results")
        })

    return model


def make(config):
    # Make the model
    model = MODELS[config.model](config.NUM_LAYERS, 
                                config.in_dim, 
                                config.HIDDEN_DIM, 
                                config.out_dim).model

    random_model = MLP(config.NUM_LAYERS, 
                        config.in_dim, 
                        config.HIDDEN_DIM, 
                        config.out_dim).model

    # Make the data
    data = generate_data(config.function,
                         config.num_train, 
                         config.num_test, 
                         config.dim, 
                         config.num_sum, 
                         config.RANGE)
    
    # Make the loss and optimizer
    criterion = F.mse_loss
    optimizer = torch.optim.RMSprop(model.parameters(), lr=config.LEARNING_RATE)

    # return random_model, model, criterion, optimizer
    return random_model, model, data, criterion, optimizer


def generate_data(function, num_train, num_test, dim, num_sum, support):

    fn = ARITHMETIC_FUNCTIONS[function]
    data = torch.FloatTensor(dim).uniform_(*support).unsqueeze_(1)
    X, y = [], []
    for i in range(num_train + num_test):
        idx_a = random.sample(range(dim), num_sum)
        idx_b = random.sample([x for x in range(dim) if x not in idx_a], num_sum)
        a, b = data[idx_a].sum(), data[idx_b].sum()
        X.append([a, b])
        y.append(fn(a, b))
    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y).unsqueeze_(1)
    indices = list(range(num_train + num_test))
    np.random.shuffle(indices)
    X_train, y_train = X[indices[num_test:]], y[indices[num_test:]]
    X_test, y_test = X[indices[:num_test]], y[indices[:num_test]]

    return ((X_train, y_train), (X_test, y_test))


def train(model, train_data, criterion, optimizer, num_iters):
    wandb.watch(model, criterion, log="all", log_freq=1000)

    data, target = train_data
    # model.train()
    for i in range(num_iters):
        out = model(data)
        loss = criterion(out, target)
        mea = torch.mean(torch.abs(target - out))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            train_log(loss, mea, i, num_iters)

def train_log(loss, mea, epoch, num_iters):
    wandb.log({"epoch": epoch, "loss_(mse)": loss.item(), 'mean_absolute_error': mea.item()})
    print("\t{}/{}: loss: {:.7f} - mea: {:.7f}".format(epoch+1, num_iters, loss.item(), mea.item()))


def test(model, test_data):
    data, target = test_data
    with torch.no_grad():
        out = model(data)
        return torch.abs(target - out)